In [1]:
import pdfplumber
import pandas as pd
import re
import requests
import tempfile
import os
import time
from unsloth import FastLanguageModel
import torch
import warnings
import traceback
from pathlib import Path
from typing import Optional, List, Tuple

warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0209 03:49:37.654000 1256 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
[tensorflow|WARNING]From c:\Users\ЛЕША\ai_challange\venv\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
def clean_text(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F]', '', text)
    text = re.sub(r'emdash\.cyr|guillemot(left|right)\.cyr', '—', text)
    text = re.sub(r'«|»|„|“', '"', text)
    text = re.sub(r'–|—', '-', text)
    text = re.sub(r'·', '.', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [3]:
def split_into_problems(full_text: str) -> list[tuple[str, str]]:
    patterns = [
        r'(\d+\.\d+\.)',
    ]
    
    all_splits = []
    for pat in patterns:
        parts = re.split(pat, full_text, flags=re.IGNORECASE)
        if len(parts) > 1:
            for i in range(1, len(parts), 2):
                num = parts[i].strip()
                content = parts[i+1].strip()
                if len(content) > 80:
                    all_splits.append((num, content))
    
    if not all_splits:
        paras = [p.strip() for p in re.split(r'\n\s*\n', full_text) if len(p.strip()) > 200]
        for i, p in enumerate(paras):
            all_splits.append((f"Auto_{i+1}", p))

    return all_splits

In [4]:
def get_prompt(text_chunk: str) -> str:
    return f"""<|im_start|>system
Ты — эксперт по олимпиадным задачам. Раздели текст строго на ТРИ блока в указанном порядке:

1. УСЛОВИЕ: Весь текст ДО первого вхождения слов "Ответ" или "Решение" (в любом регистре). Включай все данные задачи.
2. РЕШЕНИЕ: Текст, начинающийся со слов "Решение", "Первое решение", "Второе решение" и т.д. Если таких слов нет — "Нет".
3. ОТВЕТ: ТОЛЬКО если в тексте есть слово "Ответ" (с точкой, двоеточием или пробелом после), то первое законченное предложение после него.

ФОРМАТ ВЫВОДА (обязательно соблюдать):
- УСЛОВИЕ: [текст]
- РЕШЕНИЕ: [текст или "Нет"]
- ОТВЕТ: [текст или "Нет"]

ПРАВИЛА:
- НИКАКИХ пояснений, комментариев, замечаний после трёх блоков
- НИКАКИХ дополнительных строк после блока "ОТВЕТ"
- Если ответ состоит ТОЛЬКО из слова "Нет" — замени на "не может"
- Если в предложенном тебе тексте нет слова Ответ, то в поле ОТВЕТ необходимо указать "Нет".
<|im_end|>
<|im_start|>user
Текст для разбора:
{text_chunk}<|im_end|>
<|im_start|>assistant
"""

def parse_task_with_llm(text_chunk: str, tokenizer, model) -> tuple[str, str, str]:
    # Очистка от критериев оценивания
    text_chunk = re.split(
        r'\n\s*Комментарий\.|\n\s*Баллы[,\.]|\n\s*Замечание\.|\n\s*Только ответ',
        text_chunk,
        maxsplit=1,
        flags=re.IGNORECASE
    )[0].strip()
    
    if len(text_chunk) < 50:
        return "Нет", "Нет", "Нет"
    
    prompt = get_prompt(text_chunk)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Stop-критерий: остановка на двойном переносе строки
    from transformers import StoppingCriteria, StoppingCriteriaList
    
    class StopOnDoubleNewline(StoppingCriteria):
        def __call__(self, input_ids, scores, **kwargs):
            if input_ids.shape[1] < 2:
                return False
            last_tokens = input_ids[0, -2:].tolist()
            return last_tokens == [198, 198]  # \n\n в токенах Qwen
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.0,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=StoppingCriteriaList([StopOnDoubleNewline()]),
    )
    
    # Декодируем ТОЛЬКО сгенерированную часть
    result = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    ).strip()
    
    # Принудительная обрезка после блока ОТВЕТ
    if "- ОТВЕТ:" in result:
        result = result.split("- ОТВЕТ:", 1)[0] + "- ОТВЕТ:" + result.split("- ОТВЕТ:", 1)[1].split("\n\n")[0]
    # Парсинг блоков
    task_match = re.search(r'УСЛОВИЕ:\s*(.*?)(?=\s*-?\s*РЕШЕНИЕ:|$)', result, re.DOTALL | re.IGNORECASE)
    solution_match = re.search(r'РЕШЕНИЕ:\s*(.*?)(?=\s*-?\s*ОТВЕТ:|$)', result, re.DOTALL | re.IGNORECASE)
    answer_match = re.search(r'ОТВЕТ:\s*(.*?)(?=\s*-?\s*УСЛОВИЕ:|\s*-?\s*РЕШЕНИЕ:|\n\n|$)', result, re.DOTALL | re.IGNORECASE)
    
    task = (task_match.group(1).strip() if task_match else "Нет").replace('\n', ' ')
    solution = (solution_match.group(1).strip() if solution_match else "Нет").replace('\n', ' ')
    answer = (answer_match.group(1).strip() if answer_match else "Нет").replace('\n', ' ')
    
    
    return task, answer, solution  # ← ИСПРАВЛЕНО: (условие, решение, ответ)

In [5]:
def download_pdf_from_url(url: str) -> str:
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(response.content)
        tmp_file.flush()
        return tmp_file.name

def extract_text_from_pdf(pdf_path: str) -> str:
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() or "" for page in pdf.pages)
    os.unlink(pdf_path)
    return clean_text(text)

In [6]:
def process_single_pdf_to_csv(pdf_url: str, csv_path: str, subject: str, profile: str, grade: int, hardness_list: list[str], tokenizer, model):
    print(f"📥 Загрузка: {pdf_url}")
    tmp_pdf = download_pdf_from_url(pdf_url)
    full_text = extract_text_from_pdf(tmp_pdf)
    if not full_text:
        print("❌ Пустой текст")
        return pd.DataFrame()

    problems = split_into_problems(full_text)
    print(f"🔍 Найдено задач: {len(problems)}")
    results = []

    for idx, (num, content) in enumerate(problems, 1):
        if len(content.strip()) < 50:
            continue
        print(f"  → Задача {idx}: {num[:30]}...")

        if idx - 1 < len(hardness_list):
            hardness = hardness_list[idx - 1]
        else:
            hardness = hardness_list[-1] if hardness_list else "Неизвестно"

        try:
            task, answer, solution = parse_task_with_llm(content, tokenizer, model)
        except Exception as e:
            print(f"    ⚠️ Ошибка: {e}")
            task, answer, solution = "Нет", "Нет", "Нет"

        results.append({
            "subject": subject,
            "profile": profile,
            "grade": grade,
            "hardness": hardness,
            "task_text": task.replace('\n', '\\n').replace('"', "'"),
            "task_answer": answer.replace('\n', '\\n').replace('"', "'"),
            "task_solution": solution.replace('\n', '\\n').replace('"', "'"),
            "source_pdf": os.path.basename(pdf_url.split('/')[-1])
        })

    df = pd.DataFrame(results)
    
    # Добавляем данные в конец файла (заголовок только при первом создании)
    file_exists = os.path.exists(csv_path) and os.path.getsize(csv_path) > 0
    df.to_csv(
        csv_path,
        mode='a',  # ← КЛЮЧЕВОЕ ИЗМЕНЕНИЕ: режим добавления
        header=not file_exists,  # заголовок только если файл новый/пустой
        index=False,
        encoding='utf-8-sig',
        quoting=2,
        escapechar='\\'
    )
    
    print(f"✅ Добавлено {len(results)} задач в {csv_path}")
    return df

In [7]:
MODEL_NAME = "unsloth/Qwen3-4B-Instruct-2507"
MAX_SEQ_LENGTH = 8192
DTYPE = None
LOAD_IN_4BIT = True

In [8]:
print("🚀 Загрузка модели через Unsloth...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
)
FastLanguageModel.for_inference(model)
print("✅ Модель готова.")

🚀 Загрузка модели через Unsloth...
==((====))==  Unsloth 2026.1.4: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.10.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Модель готова.


In [9]:
df_empty = pd.DataFrame(columns=[
    "subject",
    "profile",
    "grade",
    "hardness",        # ← новая колонка для сложности
    "task_text",
    "task_answer",
    "task_solution",
    "source_pdf"
])

# Сохраняем в CSV
csv_path = "tasks.csv"
df_empty.to_csv(
    csv_path,
    index=False,
    encoding='utf-8-sig',
    quoting=2,          # QUOTE_NONNUMERIC
    escapechar='\\'
)

print(f"✅ Пустой файл создан: {csv_path}")
print(f"Колонки: {', '.join(df_empty.columns)}")

✅ Пустой файл создан: tasks.csv
Колонки: subject, profile, grade, hardness, task_text, task_answer, task_solution, source_pdf


In [11]:
for i in range(9, 12):
    for j in range(16, 26):
        for k in range(1, 3):
            if j == 25:
                PDF_URL = f"https://vos.olimpiada.ru/upload/files/Arhive_tasks/2024-25/final/math/sol-math-{i}-final-24-25.pdf"
            if j in [23, 24] or j == 21:
                PDF_URL = f"https://vos.olimpiada.ru/upload/files/Arhive_tasks/20{j - 1}-{j}/final/math/sol-math-{i}-day{k}-final-{j - 1}-{j}.pdf"
            if j == 22:
                PDF_URL = f"https://vos.olimpiada.ru/upload/files/Arhive_tasks/2021-22/final/math/sol-math-{i}-tur{k}-final-21-22.pdf"
            if j in [18, 19]:
                PDF_URL = f"https://vos.olimpiada.ru/upload/files/Arhive_tasks/20{j - 1}-{j}/final/math/72-ans-math-{i}-final-{j - 1}-{j % 10}.pdf"
            if j == 17:
                PDF_URL = f"https://vos.olimpiada.ru/upload/files/Arhive_tasks/2016-17/final/math/ans-math-{i}-tur{k}-final-16-7.pdf"
            if j == 16:
                PDF_URL = f"https://vos.olimpiada.ru/upload/files/Arhive_tasks/2015-16/final/math/ans-math-{i}-day{k}-final-15-6.pdf"
            CSV_PATH = "tasks.csv"
            SUBJECT = "Математика"
            PROFILE = "Математика"
            GRADE = i

            # Список сложностей: n-ая задача → n-ый элемент списка
            HARDNESS_LIST = [6, 7, 8, 9, 7, 8, 9, 10]
            try:
                df = process_single_pdf_to_csv(
                    pdf_url=PDF_URL,
                    csv_path=CSV_PATH,
                    subject=SUBJECT,
                    profile=PROFILE,
                    grade=GRADE,
                    hardness_list=HARDNESS_LIST,  # ← передаём список
                    tokenizer=tokenizer,
                    model=model
                )

                if not df.empty:
                    print(f"\n🎉 Обработано задач: {len(df)}")
                    display(df.head())
                else:
                    print("⚠️ Ни одна задача не была обработана.")
            except Exception:
                print("не палучилось", i, j, k)
                continue

📥 Загрузка: https://vos.olimpiada.ru/upload/files/Arhive_tasks/2015-16/final/math/ans-math-9-day1-final-15-6.pdf
🔍 Найдено задач: 6
  → Задача 1: 9.1....
  → Задача 2: 9.2....
  → Задача 3: 9.3....
  → Задача 4: 9.4....
  → Задача 5: 10.1....
  → Задача 6: 10.2....
✅ Добавлено 6 задач в tasks.csv

🎉 Обработано задач: 6


,subject,profile,grade,hardness,task_text,task_answer,task_solution,source_pdf
0,Математика,Математика,9,6,У менялы на базаре есть много ковров. Он согла...,Обманывает,"Назовём ковер, все стороны которого больше 1, ...",ans-math-9-day1-final-15-6.pdf
1,Математика,Математика,9,7,Окружность ω касается сторон угла BAC в точках...,Нет,"Если (cid:96) (cid:107) BC, утверждение очевид...",ans-math-9-day1-final-15-6.pdf
2,Математика,Математика,9,8,Саша выбрал натуральное число N > 1 и выписал ...,N = 3,"Заметим сразу, что dᵢ = N/dᵢ при всех i = s+1−...",ans-math-9-day1-final-15-6.pdf
3,Математика,Математика,9,9,Из клетчатого бумажного квадрата 100×100 вырез...,Нет,"Первое решение. Представим себе, что доминошки...",ans-math-9-day1-final-15-6.pdf
4,Математика,Математика,9,7,В Национальной Баскетбольной Ассоциации 30 ком...,Нет,"Пусть x и y — общее число матчей, сыгранных вн...",ans-math-9-day1-final-15-6.pdf


📥 Загрузка: https://vos.olimpiada.ru/upload/files/Arhive_tasks/2015-16/final/math/ans-math-9-day2-final-15-6.pdf
🔍 Найдено задач: 8
  → Задача 1: 9.5....
  → Задача 2: 9.109....
  → Задача 3: 9.6....
  → Задача 4: 9.7....
  → Задача 5: 9.8....
  → Задача 6: 10.5....
  → Задача 7: 9.109....
  → Задача 8: 10.6....
✅ Добавлено 8 задач в tasks.csv

🎉 Обработано задач: 8


,subject,profile,grade,hardness,task_text,task_answer,task_solution,source_pdf
0,Математика,Математика,9,6,"Из цифр 1, 2, 3, 4, 5, 6, 7, 8, 9 составлены д...",8,"Покажем, что сумма не может оканчиваться на 9 ...",ans-math-9-day2-final-15-6.pdf
1,Математика,Математика,9,7,"Значит, одно из них не меньше 109, что невозмо...",Нет,Нет,ans-math-9-day2-final-15-6.pdf
2,Математика,Математика,9,8,Квадрат разбит на n² прямоугольников 2(n−1) пр...,Нет,"Назовём пару прямоугольников вложимой, если од...",ans-math-9-day2-final-15-6.pdf
3,Математика,Математика,9,9,"Окружность ω вписана в треугольник ABC, в кото...",сумма XY + XZ не зависит от выбора точки X.,"Будем считать, что точка Y лежит ближе к точке...",ans-math-9-day2-final-15-6.pdf
4,Математика,Математика,9,7,"Сумма положительных чисел a, b, c и d равна 3....",Нет,"Домножив доказываемое неравенство на a2b2c2d2,...",ans-math-9-day2-final-15-6.pdf


📥 Загрузка: https://vos.olimpiada.ru/upload/files/Arhive_tasks/2016-17/final/math/ans-math-9-tur1-final-16-7.pdf
🔍 Найдено задач: 4
  → Задача 1: 9.1....
  → Задача 2: 9.2....
  → Задача 3: 9.3....
  → Задача 4: 9.4....
✅ Добавлено 4 задач в tasks.csv

🎉 Обработано задач: 4


...

Unsloth: Input IDs of shape torch.Size([1, 13700]) with length 13700 > the model's max sequence length of 8192.
We shall truncate it ourselves. It's imperative if you correct this issue first.


  → Задача 16: 11.8....
✅ Добавлено 16 задач в tasks.csv

🎉 Обработано задач: 16


,subject,profile,grade,hardness,task_text,task_answer,task_solution,source_pdf
0,Математика,Математика,11,6,"(M) Если есть хотя бы два из недочётов M1, M2,...",Нет,Нет,sol-math-11-final-24-25.pdf
1,Математика,Математика,11,7,За использование без доказательства равенства ...,Нет,Нет,sol-math-11-final-24-25.pdf
2,Математика,Математика,11,8,"(0) Не оценивается. (0.1) Утверждение, что важ...",Нет,Нет,sol-math-11-final-24-25.pdf
3,Математика,Математика,11,9,"(1) Пример хуже оптимального (например, на (2N...",Нет,Нет,sol-math-11-final-24-25.pdf
4,Математика,Математика,11,7,(1) Только оценка сверху - 2 балла. (2) Только...,Нет,Нет,sol-math-11-final-24-25.pdf


📥 Загрузка: https://vos.olimpiada.ru/upload/files/Arhive_tasks/2024-25/final/math/sol-math-11-final-24-25.pdf
🔍 Найдено задач: 16
  → Задача 1: 11.1....
  → Задача 2: 11.2....
  → Задача 3: 11.3....
  → Задача 4: 11.4....
  → Задача 5: 11.5....
  → Задача 6: 11.6....
  → Задача 7: 11.7....
  → Задача 8: 11.8....
  → Задача 9: 11.1....
  → Задача 10: 11.2....
  → Задача 11: 11.3....
  → Задача 12: 11.4....
  → Задача 13: 11.5....
  → Задача 14: 11.6....
  → Задача 15: 11.7....
  → Задача 16: 11.8....
✅ Добавлено 16 задач в tasks.csv

🎉 Обработано задач: 16


,subject,profile,grade,hardness,task_text,task_answer,task_solution,source_pdf
0,Математика,Математика,11,6,"(M) Если есть хотя бы два из недочётов M1, M2,...",Нет,Нет,sol-math-11-final-24-25.pdf
1,Математика,Математика,11,7,За использование без доказательства равенства ...,Нет,Нет,sol-math-11-final-24-25.pdf
2,Математика,Математика,11,8,"(0) Не оценивается. (0.1) Утверждение, что важ...",Нет,Нет,sol-math-11-final-24-25.pdf
3,Математика,Математика,11,9,"(1) Пример хуже оптимального (например, на (2N...",Нет,Нет,sol-math-11-final-24-25.pdf
4,Математика,Математика,11,7,(1) Только оценка сверху - 2 балла. (2) Только...,Нет,Нет,sol-math-11-final-24-25.pdf


In [12]:
# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2023-24/final/math/sol-math-9-day1-final-23-24.pdf
# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2022-23/final/math/sol-math-9-day1-final-22-23.pdf


# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2021-22/final/math/sol-math-9-tur1-final-21-22.pdf


# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2020-21/final/math/sol-math-9-day1-final-20-21.pdf


# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2018-19/final/math/72-ans-math-9-final-18-9.pdf
# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2017-18/final/math/72-ans-math-9-final-17-8.pdf


# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2016-17/final/math/ans-math-9-tur1-final-16-7.pdf


# https://vos.olimpiada.ru/upload/files/Arhive_tasks/2015-16/final/math/ans-math-9-day1-final-15-6.pdf

In [13]:
data = {
    "графы" : {
        "лемма хола",
        "двудольность + паросочетания",
        "теорема турана"
    },
    "геометрия" : {
        "счет" : [
            "счет в синусах",
            "обычный счет углов",
            "счет в отрезках",
            "счет в комплах",
            "счет в координатах"
        ],
        "неравенства" : [
            "неравенства"
        ],
        "свойства окружностей" : [
            "степень точки и радикальная ось/центр",
            "именные теоремы в окружностях",
            "изогональное сопряжение",
            "вписанность/описанность"
        ],
        "стереометрия" : {
            "задачи со сферами",
            "задачи без сфер"
        },
    },
    "теория чисел" : {
        "функции" : [
            "теорема безу",
            "основная теорема алгебры",
            "теорема виета и свойства корней",
            "графические задачи",
            "прогрессии",
            "немногочлены",
            "функциональные уравнения"
        ],
        "неравенства" : [
            "КБШ и лемма Титту",
            "неравенство о средних",
            "неравенство Йенсенна",
        ],
        "делимоость" : [
            "МТФ, теорема Эйлера",
            "функция Эйлера",
            "НОД, НОК"
        ]
    },
    "комбинаторика" : {
        "игры" : [
            "стратегия за двоих",
            "стратегия + оценка",
            "рыцари и лжецы"
        ],
        "оценка + пример" : [
            "на клеточках",
            "не на клеточках"
        ],
        "шахматы" : [
            "задачи на шахматы",
            "задачи на шахматную раскраску"
        ]
    }
}

In [14]:
import pandas as pd

df = pd.read_csv('tasks.csv')
# Удаляем дубликаты по ключевым полям, оставляя последнюю запись
df = df.drop_duplicates(subset=['task_text'], keep='last')

In [15]:
import re

# Замена (cid:n) на пробелы
df['task_text'] = df['task_text'].apply(lambda x: re.sub(r'\(cid:\d+\)', '', str(x)))
df['task_answer'] = df['task_answer'].apply(lambda x: re.sub(r'\(cid:\d+\)', '', str(x)))
df['task_solution'] = df['task_solution'].apply(lambda x: re.sub(r'\(cid:\d+\)', '', str(x)))

# Замена ◦ на °
df['task_text'] = df['task_text'].str.replace('◦', '°')
df['task_solution'] = df['task_solution'].str.replace('◦', '°')

# Исправление ё (если нужно)
df['task_text'] = df['task_text'].str.replace('ё', 'ё')
df['task_answer'] = df['task_answer'].str.replace('ё', 'ё')
df['task_solution'] = df['task_solution'].str.replace('ё', 'ё')

In [16]:
df.to_csv("final.csv", index=False)

In [ ]:
import pandas as pd
from unsloth import FastLanguageModel
import torch
import re
import ast

# --- 1. Загрузка модели ---
MODEL_NAME = "unsloth/Qwen3-4B-Instruct-2507"
max_seq_length = 8192

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)

In [ ]:
data = {
    "графы": {
        "лемма хола": [],
        "двудольность + паросочетания": [],
        "теорема турана": []
    },
    "геометрия": {
        "счет": [
            "счет в синусах",
            "счет углов",
            "счет в отрезках",
            "счет в комплах",
            "счет в координатах"
        ],
        "неравенства": [
            "неравенства"
        ],
        "свойства окружностей": [
            "степень точки и радикальная ось/центр",
            "именные теоремы в окружностях",
            "изогональное сопряжение",
            "вписанность/описанность"
        ],
        "стереометрия": [
            "задачи со сферами",
            "задачи без сфер"
        ],
        "площадь+периметр" : [
            "площадь+периметр"
        ]
    },
    "теория чисел": {
        "функции": [
            "теорема безу",
            "основная теорема алгебры",
            "теорема виета и свойства корней",
            "графические задачи",
            "прогрессии",
            "немногочлены",
            "функциональные уравнения"
        ],
        "неравенства": [
            "КБШ и лемма Титту",
            "неравенство о средних",
            "неравенство Йенсенна",
            "другое"
        ],
        "делимоость": [
            "МТФ, теорема Эйлера",
            "функция Эйлера",
            "НОД, НОК"
        ]
    },
    "комбинаторика": {
        "игры": [
            "стратегия за двоих",
            "стратегия + оценка",
            "рыцари и лжецы"
        ],
        "оценка + пример": [
            "на клеточках",
            "не на клеточках"
        ],
        "шахматы": [
            "задачи на шахматы",
            "задачи на шахматную раскраску"
        ],
        "комбигеома" : [
            "комбипланиметрия",
            "комбистереометрия"
        ]
    }
}

# --- 3. Функция для генерации списка всех допустимых тем ---
def generate_theme_list(theme_dict):
    theme_list = []
    for section, topics in theme_dict.items():
        if isinstance(topics, dict):
            for topic, subtopics in topics.items():
                if isinstance(subtopics, list) and subtopics:
                    for subtopic in subtopics:
                        theme_list.append(f"{section} | {topic} | {subtopic}")
                else: # Если подтем нет или она не определена, используем тему как подтему
                    theme_list.append(f"{section} | {topic} | {topic}")
        else: # Если для раздела напрямую указан список (как в "графы")
            if isinstance(topics, list) and topics:
                 for topic in topics:
                     theme_list.append(f"{section} | {topic} | {topic}")
            else:
                 theme_list.append(f"{section} | {section} | {section}")
    return theme_list

all_themes = generate_theme_list(data)
themes_str_for_prompt = "\n".join(all_themes)
print("--- Допустимые темы для классификации ---")
for t in all_themes[:10]: # Показываем первые 10
    print(t)
if len(all_themes) > 10:
    print("...")
print(f"Всего уникальных комбинаций тем: {len(all_themes)}")
print("------------------------------------------\n")

# --- 4. Функция для получения ответа от LLM ---
def get_llm_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=200,
        temperature=0.01, # Минимальная температура для детерминированного вывода
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    return response.strip()

# --- 5. Функция классификации ---
def classify_theme(condition_text, solution_text, themes_list_str):
    # Объединяем условие и решение для анализа
    full_context = f"Условие задачи:\n{condition_text}\n\nРешение задачи:\n{solution_text}"
    
    prompt = f"""Ты - эксперт по классификации математических задач.
Прочитай условие и решение задачи и выбери наиболее подходящую тему из строго заданного списка.
Ответь ТОЛЬКО в формате "Раздел | Тема | Подтема", без объяснений и без создания новых тем.
Если подходящей темы нет, ответь "другое | другое | другое".

Список допустимых тем:
{themes_list_str}

Условие и решение задачи:
{full_context[:2000]} # Ограничиваем длину контекста для экономии памяти и времени

Ответ:"""
    
    max_attempts = 3
    for attempt in range(max_attempts):
        try:
            response = get_llm_response(prompt)
            print(f"  -> Ответ LLM (попытка {attempt+1}): {response}") # Отладочный вывод
            
            # Проверяем, содержит ли ответ разделитель |
            if " | " in response:
                parts = response.split(" | ")
                if len(parts) == 3:
                    selected_theme_key = response
                    # Проверяем, есть ли такая комбинация в списке
                    if selected_theme_key in themes_list_str:
                        print(f"  -> Выбрана валидная тема: {selected_theme_key}")
                        return parts[0], parts[1], parts[2]
                    else:
                        print(f"  -> Ответ '{selected_theme_key}' не найден в списке допустимых тем.")
                else:
                    print(f"  -> Ответ не соответствует формату 'Раздел | Тема | Подтема'.")
            else:
                print(f"  -> Ответ не содержит разделитель ' | '.")
            
        except Exception as e:
            print(f"  -> Ошибка при классификации (попытка {attempt+1}): {e}")
    
    print(f"  -> Не удалось получить валидный ответ после {max_attempts} попыток. Используем 'другое'.")
    return "другое", "другое", "другое"


# --- 6. Основная функция обработки CSV ---
def process_csv(input_file, output_file, theme_data):
    df = pd.read_csv(input_file)
    print(f"Загружен CSV файл с {len(df)} строками и колонками: {list(df.columns)}")
    
    # Определяем колонки
    condition_col = 'task_text' # Условие задачи
    solution_col = 'task_solution'  # Решение задачи
    answer_col = 'task_answer'      # Ответ задачи
    
    if condition_col not in df.columns:
        raise ValueError(f"Колонка '{condition_col}' не найдена в CSV файле.")
    if solution_col not in df.columns:
        raise ValueError(f"Колонка '{solution_col}' не найдена в CSV файле.")
    if answer_col not in df.columns:
        raise ValueError(f"Колонка '{answer_col}' не найдена в CSV файле.")

    # Создаем новые колонки
    df['section'] = ""
    df['topic'] = ""
    df['subtopic'] = ""
    df['has_answer'] = False
    
    all_themes_list = generate_theme_list(theme_data)
    themes_str = "\n".join(all_themes_list)
    
    for idx, row in df.iterrows():
        print(f"\n--- Обработка строки {idx + 1}/{len(df)} ---")
        condition_text = str(row[condition_col])
        solution_text = str(row[solution_col]) # Используем решение
        
        # Определяем флаг наличия ответа
        answer_value = str(row[answer_col])
        has_answer = answer_value != "Нет" and pd.notna(row[answer_col]) and answer_value != ""
        df.at[idx, 'has_answer'] = has_answer
        print(f"  Условие: {condition_text[:100]}...") # Показываем начало условия
        print(f"  Решение: {solution_text[:100]}...") # Показываем начало решения
        print(f"  Ответ: {answer_value}, has_answer: {has_answer}")
        
        # Классифицируем тему на основе УСЛОВИЯ И РЕШЕНИЯ
        sec, top, subt = classify_theme(condition_text, solution_text, themes_str)
        
        df.at[idx, 'section'] = sec
        df.at[idx, 'topic'] = top
        df.at[idx, 'subtopic'] = subt
        
        print(f"  -> Результат: {sec} | {top} | {subt}")

    # Сохраняем результат
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\n--- Обработка завершена. Результат сохранен в {output_file} ---")

# --- 7. Запуск обработки ---
if __name__ == "__main__":
    input_file = "final.csv"  # Входной CSV файл
    output_file = "final_with_themes_final.csv"  # Выходной CSV файл
    
    process_csv(input_file, output_file, data)

--- Допустимые темы для классификации ---
графы | лемма хола | лемма хола
графы | двудольность + паросочетания | двудольность + паросочетания
графы | теорема турана | теорема турана
геометрия | счет | счет в синусах
геометрия | счет | обычный счет углов
геометрия | счет | счет в отрезках
геометрия | счет | счет в комплах
геометрия | счет | счет в координатах
геометрия | неравенства | неравенства
геометрия | свойства окружностей | степень точки и радикальная ось/центр
...
Всего уникальных комбинаций тем: 39
------------------------------------------

Загружен CSV файл с 161 строками и колонками: ['subject', 'profile', 'grade', 'hardness', 'task_text', 'task_answer', 'task_solution', 'source_pdf']

--- Обработка строки 1/161 ---
  Условие: У менялы на базаре есть много ковров. Он согласен взамен ковра размера a×b дать либо ковер размера a...
  Решение: Назовём ковер, все стороны которого больше 1, большим, а ковер, все стороны которого меньше 1, — мал...
  Ответ: Обманывает, has_answer: 

In [6]:
import pandas as pd
import glob

# Список файлов для склеивания
files = ['final_with_themes_final.csv', 'reg_with_themes_final.csv', 'mun_with_themes_final.csv']  # замените на ваши имена

# Читаем все файлы и объединяем
dfs = [pd.read_csv(f) for f in files]
combined = pd.concat(dfs, ignore_index=True)

# Сохраняем результат
combined.to_csv('tasks.csv', index=False)
print(f"Склеено {len(combined)} строк в tasks.csv")

Склеено 446 строк в tasks.csv
